# Олимпиада НТИ - как выиграть за 30 строк

![](https://pp.userapi.com/c638719/v638719005/2f6da/mFHY83I9u2k.jpg)

### Машинное обучение и анализ данных - задача №2



Выделение из текста заранее известных сущностей - одна из самых распространенных

Середина двадцатого века один из самых сложных периодов нашей страны. По разным причинам (на войне, или в результате репрессий) пропало множество людей, о которых практически не осталось данных. Все данные которые мы можем выяснить это скупые карточки и журнальные записи, оставленные в разных формах. Множество общественных организаций занимается оцифровкой этих данных, чтобы наши соотечественники могли узнать судьбу своей семьи.   Однако, как писалось ранее, эти данные представляют собой скупые  и не структурированные заметки из разных журналов и форм, для того чтобы люди могли найти своих родственников необходимо из этих заметок выделить как можно больше классифицированной информации (сущностей)

Необходимо осуществить выделение сущностей из коротких текстов карточек. При этом не все записи содержат все сущности. Задача называется named entity recognition.
Список сущностей

Имя, фамилия, отчество, "никнейм или кличка", пол, год рождения, место рождения, работы, место проживания, был-ли арстован, день ареста, месяц ареста, год ареста,  кем арестован,  день суда, месяц суда, год суда, статья обвинения, наказание, был-ли растрелян, день расстрела, месяц расстрела, год расстрела, день реабилитации, месяц реабилитации, год реабилитации, источник, были ли дети, сам текст.


В качестве обучающей выборки представлены уже разобранные варианты.

# Решение

Задача состоит из трёх датасетов. Давайте их скачаем.

    Важно! Датасет из которого мы будем генерировать ответ нельзя использовать для вытаскивания фичей. На олимпиде выдавали кусок датасета на 5 минут, чтобы ты применил к нему свой код, а не руками доставал значения.

In [ ]:
# Датасет на котором можно обучаться
!wget https://tvorog.me/task2_train.csv

In [1]:
import re
import pandas as pd

In [2]:
# Загружаем данные в pandas
data = pd.DataFrame.from_csv("task2_train.csv")

In [8]:
data.head() # посмотрим на них

,id,lastname,firstname,middlename,alternative_name,sex,birthyear,birth_place,job,liveplace,...,is_shooted,shooted_day,shooted_month,shooted_year,rehabilitation_day,rehabilitation_month,rehabilitation_year,source,had_children,text
38123,111202,Пекин,Михаил,Иванович,NaN,м,1894,г.Казань,"техник-химик, завод СК-4.",г.Казань,...,Да,3.0,1.0,1938.0,28.0,6.0,1989.0,КП Республики Татарстан,NaN,"НИКИТИН Иван Алексеевич, 1922 г.р., место рожд..."
41651,114494,Савельев,Иван,Михайлович,NaN,м,1879,"Лаишевский р-н, с.Никольское","середняк, церковный староста.","Лаишевский р-н, с.Никольское",...,NaN,NaN,NaN,NaN,5.0,6.0,1991.0,КП Республики Татарстан,NaN,"Савельев Иван Михайлович, 1879 г.р., место рож..."
45583,118829,Сульг,Адам,Иоаннович,| |Иоганнович,м,1896,"Лифляндская губ., Веровский у., с.Корыстово",дежурный по ст.,"Казанская ж.д., ст.Бобыльская.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,КП Республики Татарстан,NaN,"Сульг Адам Иоаннович (Иоганнович), 1896 г.р., ..."
5036,401513,Маликов,Иосиф,Миронович,NaN,м,1871,х. Хворостяной Яковлевского р-на,крестьянин.,х. Хворостяной Яковлевского р-на,...,NaN,NaN,NaN,NaN,23.0,2.0,2000.0,КП Белгородской обл.,NaN,"Маликов Иосиф Миронович, 1871 г.р., ур. х. Хво..."
31127,107308,Моряшов,Егор,Степанович,NaN,м,1887,"Арский р-н, п.Мокрый Луг (?)",Крестьянин.,"Арский р-н, п.Мокрый Луг (?)",...,NaN,NaN,NaN,NaN,4.0,9.0,1989.0,КП Республики Татарстан,NaN,"Моряшов Егор Степанович, 1887 г.р., место рожд..."


# Методы решения
[Regexp](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F) это то, что вы ищите!

Далее будут примеры того, как доставать поля.

In [9]:
get_years = lambda text: re.findall('[0-9]{1,2}.[0-9]{1,2}.[0-9]{2,4}', text)

In [20]:
years = data['text'].apply(get_years)

In [21]:
years[:10]

38123                           [24.2.43, 15.6.43, 9.6.92]
41651                 [4.11.28, 24.11.28, 10.5.29, 5.6.91]
45583                           [21.2.38, 15.6.38, 3.9.44]
5036                  [13.12.1929, 20.12.1929, 23.02.2000]
31127                           [12.3.30, 22.3.30, 4.9.89]
48180                                   [15.4.32, 17.5.32]
53582                        [23.10.37, 25.5.40, 16.10.58]
14285                                 [20.05.47, 10.05.89]
55939    [28.11.35, 4.1.36, 23.4.38, 13.2.39, 4.11.42, ...
54321                           [9.3.42, 20.6.42, 21.3.97]
Name: text, dtype: object

Иногда можно использовать не только regexp. Например, вытащить ФИО можно так:

In [15]:
get_name = lambda text: text.split()[0]

In [16]:
name = data['text'].apply(get_name)

In [18]:
name[:10]

38123           НИКИТИН
41651          Савельев
45583             Сульг
5036            Маликов
31127           Моряшов
48180           Федоров
53582           ШУЙСКИЙ
14285    КРЫЛОВА-ТЮКИНА
55939          Коноплев
54321           ЯКУПОВА
Name: text, dtype: object